<a href="https://colab.research.google.com/github/Sagi1977/CycleTrading-/blob/TEST/Cycle_Trades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *** Base APP !!!***

```
# This is formatted as code
```



In [1]:
# Install necessary libraries
!pip install pandas_ta
import yfinance as yf
import pandas as pd
import pandas_ta as ta
from datetime import datetime # Import datetime for getting current time

# --- 1. Define Assets ---
ASSET_GROUPS = {
    "ETF": ['SPY', 'QQQ', 'MAGS', 'IBIT', 'ETHA'],
    "STOCK": ['AAPL', 'MSFT', 'GOOGL', 'META', 'AMZN', 'NVDA', 'TSLA']
}

# --- 2. Define Trading Days (Adjusted for Short/Medium Term) ---
N_DAYS_SELECTED = 150 # Focusing on short to medium term trends

# --- 3. Function to Fetch Base Data ---
def get_base_data(ticker, days):
    """
    Fetches historical data (Open, Low, High, Close, Volume) for a given ticker.
    """
    try:
        # Fetch enough data to ensure the selected number of days are available
        # Fetching 1 year should be sufficient for 150 days
        df = yf.download(ticker, period="1y", progress=False)
        if df.empty:
            return pd.DataFrame()
    except Exception:
        return pd.DataFrame()

    # Keep only the required columns
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df_final = df[['Open', 'Low', 'High', 'Close', 'Volume']].tail(days)

    return df_final

# --- Function to Fetch Current Price (for Crypto) ---
def get_current_price(ticker):
    """
    Fetches the current price for a given ticker.
    Uses yfinance for consistency, may return delayed data.
    For true real-time, a crypto-specific API would be needed.
    """
    try:
        # Fetch a very short period to get the latest data point
        ticker_info = yf.Ticker(ticker)
        # Get the most recent price data
        latest_data = ticker_info.history(period="1d", interval="1m")
        if not latest_data.empty:
            return latest_data['Close'].iloc[-1]
        else:
            # Fallback: try fetching info
             info = ticker_info.info
             if 'currentPrice' in info:
                 return info['currentPrice']
             return None
    except Exception:
        return None

# --- Function to Fetch VIX Data ---
def get_vix_data(days):
    """
    Fetches historical Close data for the VIX (^VIX) ticker.
    """
    try:
        df = yf.download('^VIX', period="1y", progress=False)
        if df.empty:
            return pd.DataFrame()
    except Exception:
        return pd.DataFrame()

    # Keep only the 'Close' column and rename it
    df_vix = df[['Close']].copy()
    df_vix.columns = ['VIX_Close']

    # Return the last 'days' rows
    return df_vix.tail(days)


# --- 4. Fetch Data and Calculate Indicators ---
fetched_data = {}
analyzed_data = {}

print("=========================================================")
print(f"        Fetching Data and Calculating Indicators ({N_DAYS_SELECTED} days)")
print("=========================================================")

# Fetch VIX data once
df_vix = get_vix_data(N_DAYS_SELECTED)
if df_vix.empty:
    print("!!! Could not fetch VIX data. VIX-related indicators will be skipped.")
else:
    print(f"VIX data fetched. DataFrame shape: {df_vix.shape}")


for group_name, tickers in ASSET_GROUPS.items():
    print(f"\n################## Group: {group_name} ##################")
    fetched_data[group_name] = {}
    analyzed_data[group_name] = {}

    for ticker in tickers:
        print(f"\n--- Processing data for {ticker} ---")

        # Fetch data
        df_base = get_base_data(ticker, N_DAYS_SELECTED)

        if not df_base.empty:
            fetched_data[group_name][ticker] = df_base.copy() # Store fetched data

            # Calculate technical indicators using pandas_ta (Relevant for short/medium term)
            print(f"--- Calculating indicators for {ticker} ---")
            df_analysis = df_base.copy() # Work on a copy
            df_analysis.ta.rsi(length=14, append=True)
            df_analysis.ta.atr(length=14, append=True)
            df_analysis.ta.sma(length=50, append=True) # Medium-term MA
            df_analysis.ta.sma(length=100, append=True) # Longer end of medium-term MA
            df_analysis.ta.sma(length=150, append=True) # Added SMA 150
            df_analysis.ta.macd(fast=12, slow=26, signal=9, append=True)

            # Add more indicators (examples)
            df_analysis.ta.bbands(length=20, append=True) # Bollinger Bands
            df_analysis.ta.ema(length=20, append=True) # Exponential Moving Average (Short-term)
            df_analysis.ta.vwap(append=True) # Volume Weighted Average Price

            # Add Stochastic Oscillator with specified parameters (Short-term)
            stoch_k, stoch_d, stoch_smooth_k = 14, 3, 3
            df_analysis.ta.stoch(k=stoch_k, d=stoch_d, smooth_k=stoch_smooth_k, append=True)

            # --- Add Logic to Combine Stochastic with MAs (Adjusted for Medium Term) ---
            # Example: Check if Stochastic %K is above SMA 50 (Potential bullish signal relative to medium trend)
            stoch_k_col = f'STOCHk_{stoch_k}_{stoch_d}_{stoch_smooth_k}'
            sma_50_col = 'SMA_50'
            if stoch_k_col in df_analysis.columns and sma_50_col in df_analysis.columns:
                df_analysis['STOCHk_vs_SMA50_Above'] = (df_analysis[stoch_k_col] > df_analysis[sma_50_col]).astype(int)

            # Removed comparison to SMA 200 as we are focusing on shorter/medium terms

            # --- Add Fibonacci Retracement and Extension Levels (Based on Daily Data) ---
            # Calculate based on the High and Low of the current daily data range (150 days)
            fib_max_price_daily = df_analysis['High'].max()
            fib_min_price_daily = df_analysis['Low'].min()
            fib_range_daily = fib_max_price_daily - fib_min_price_daily

            # Fibonacci Retracement Levels (Daily)
            df_analysis['Fib_0_Retracement_Daily'] = fib_max_price_daily
            df_analysis['Fib_23.6_Retracement_Daily'] = fib_max_price_daily - (fib_range_daily * 0.236)
            df_analysis['Fib_38.2_Retracement_Daily'] = fib_max_price_daily - (fib_range_daily * 0.382)
            df_analysis['Fib_50.0_Retracement_Daily'] = fib_max_price_daily - (fib_range_daily * 0.50)
            df_analysis['Fib_61.8_Retracement_Daily'] = fib_max_price_daily - (fib_range_daily * 0.618)
            df_analysis['Fib_78.6_Retracement_Daily'] = fib_max_price_daily - (fib_range_daily * 0.786)
            df_analysis['Fib_100_Retracement_Daily'] = fib_min_price_daily

            # Fibonacci Extension Levels (Daily)
            df_analysis['Fib_123.6_Extension_Daily'] = fib_max_price_daily + (fib_range_daily * 0.236)
            df_analysis['Fib_138.2_Extension_Daily'] = fib_max_price_daily + (fib_range_daily * 0.382)
            df_analysis['Fib_161.8_Extension_Daily'] = fib_max_price_daily + (fib_range_daily * 0.618)
            df_analysis['Fib_200.0_Extension_Daily'] = fib_max_price_daily + (fib_range_daily * 1.00)
            df_analysis['Fib_261.8_Extension_Daily'] = fib_max_price_daily + (fib_range_daily * 1.618)


            # --- Add Fibonacci Retracement and Extension Levels (Based on Weekly Data) ---
            # Resample daily data to weekly
            df_weekly = df_base.resample('W').agg({
                'Open': 'first',
                'High': 'max',
                'Low': 'min',
                'Close': 'last',
                'Volume': 'sum'
            }).dropna() # Drop weeks with no data

            if not df_weekly.empty:
                # Calculate based on the High and Low of the current weekly data range
                fib_max_price_weekly = df_weekly['High'].max()
                fib_min_price_weekly = df_weekly['Low'].min()
                fib_range_weekly = fib_max_price_weekly - fib_min_price_weekly

                # Fibonacci Retracement Levels (Weekly)
                df_analysis['Fib_0_Retracement_Weekly'] = fib_max_price_weekly
                df_analysis['Fib_23.6_Retracement_Weekly'] = fib_max_price_weekly - (fib_range_weekly * 0.236)
                df_analysis['Fib_38.2_Retracement_Weekly'] = fib_max_price_weekly - (fib_range_weekly * 0.382)
                df_analysis['Fib_50.0_Retracement_Weekly'] = fib_max_price_weekly - (fib_range_weekly * 0.50)
                df_analysis['Fib_61.8_Retracement_Weekly'] = fib_max_price_weekly - (fib_range_weekly * 0.618)
                df_analysis['Fib_78.6_Retracement_Weekly'] = fib_max_price_weekly - (fib_range_weekly * 0.786)
                df_analysis['Fib_100_Retracement_Weekly'] = fib_min_price_weekly

                # Fibonacci Extension Levels (Weekly)
                df_analysis['Fib_123.6_Extension_Weekly'] = fib_max_price_weekly + (fib_range_weekly * 0.236)
                df_analysis['Fib_138.2_Extension_Weekly'] = fib_max_price_weekly + (fib_range_weekly * 0.382)
                df_analysis['Fib_161.8_Extension_Weekly'] = fib_max_price_weekly + (fib_range_weekly * 0.618)
                df_analysis['Fib_200.0_Extension_Weekly'] = fib_max_price_weekly + (fib_range_weekly * 1.00)
                df_analysis['Fib_261.8_Extension_Weekly'] = fib_max_price_weekly + (fib_range_weekly * 1.618)
            else:
                 print(f"!!! No sufficient weekly data for {ticker} to calculate Weekly Fibonacci levels.")

            # --- Add VIX data and VIX Regime Indicator ---
            if not df_vix.empty:
                # Merge VIX data with the current asset's data
                df_analysis = pd.merge(df_analysis, df_vix, left_index=True, right_index=True, how='left')

                # Calculate VIX Regime (simple example: above/below a threshold)
                # You might define a more sophisticated regime based on moving averages, volatility bands, etc.
                vix_threshold = 20 # Example threshold
                df_analysis['VIX_Regime_Above_Threshold'] = (df_analysis['VIX_Close'] > vix_threshold).astype(int)
                print(f"VIX data merged and VIX Regime calculated for {ticker}.")
            else:
                 print(f"!!! Skipping VIX-related indicators for {ticker} due to missing VIX data.")
            # --- End Add VIX data and VIX Regime Indicator ---


            # --- Fetch and Add Current Crypto Price ---
            if ticker == 'IBIT':
                btc_ticker = 'BTC-USD'
                current_btc_price = get_current_price(btc_ticker)
                if current_btc_price is not None:
                    # Add current price to the last row of the IBIT DataFrame
                    # Create new columns for current price and its timestamp
                    current_time = datetime.now()
                    df_analysis.loc[df_analysis.index[-1], f'Current_Price_{btc_ticker}'] = current_btc_price
                    df_analysis.loc[df_analysis.index[-1], f'Current_Price_Timestamp_{btc_ticker}'] = current_time
                    print(f"Added current price for {btc_ticker}: {current_btc_price}")
                else:
                    print(f"!!! Could not fetch current price for {btc_ticker}.")

            elif ticker == 'ETHA':
                eth_ticker = 'ETH-USD'
                current_eth_price = get_current_price(eth_ticker)
                if current_eth_price is not None:
                     # Add current price to the last row of the ETHA DataFrame
                    # Create new columns for current price and its timestamp
                    current_time = datetime.now()
                    df_analysis.loc[df_analysis.index[-1], f'Current_Price_{eth_ticker}'] = current_eth_price
                    df_analysis.loc[df_analysis.index[-1], f'Current_Price_Timestamp_{eth_ticker}'] = current_time
                    print(f"Added current price for {eth_ticker}: {current_eth_price}")
                else:
                    print(f"!!! Could not fetch current price for {eth_ticker}.")
            # --- End Fetch and Add Current Crypto Price ---


            analyzed_data[group_name][ticker] = df_analysis # Store analyzed data
            print(f"Indicators calculated for {ticker}. DataFrame shape: {df_analysis.shape}")
        else:
            print(f"!!! Skipping {ticker} as no data was available.")

print("\n=========================================================")
print("        Data Fetching and Indicator Calculation Complete ")
print("=========================================================")

# --- 5. Save Analyzed Data to Excel ---
output_filename = "analyzed_financial_data.xlsx"

print(f"\n=========================================================")
print(f"        Saving Analyzed Data to {output_filename}         ")
print("=========================================================")

with pd.ExcelWriter(output_filename) as writer:
    for group_name, group_data in analyzed_data.items():
        if group_data: # Check if there is data for the group
            # Combine all tickers within the group into a single DataFrame for saving
            # Add a 'Ticker' column to identify the data for each ticker
            combined_df = pd.concat(
                [df.assign(Ticker=ticker) for ticker, df in group_data.items()]
            )

            # Save the combined DataFrame for the group to a sheet named after the group
            combined_df.to_excel(writer, sheet_name=group_name)

            print(f"Analyzed data for {group_name} saved to sheet '{group_name}'")
        else:
            print(f"No analyzed data to save for {group_name} group.")

print("\n=========================================================")
print("        Analyzed Data Export Complete                    ")
print("=========================================================")

        Fetching Data and Calculating Indicators (150 days)


/tmp/ipython-input-1065870498.py:66: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('^VIX', period="1y", progress=False)


VIX data fetched. DataFrame shape: (150, 1)

################## Group: ETF ##################

--- Processing data for SPY ---


/tmp/ipython-input-1065870498.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)


--- Calculating indicators for SPY ---
VIX data merged and VIX Regime calculated for SPY.
Indicators calculated for SPY. DataFrame shape: (150, 50)

--- Processing data for QQQ ---


...

--- Calculating indicators for AMZN ---
VIX data merged and VIX Regime calculated for AMZN.
Indicators calculated for AMZN. DataFrame shape: (150, 50)

--- Processing data for NVDA ---


/tmp/ipython-input-1065870498.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)


--- Calculating indicators for NVDA ---
VIX data merged and VIX Regime calculated for NVDA.
Indicators calculated for NVDA. DataFrame shape: (150, 50)

--- Processing data for TSLA ---


/tmp/ipython-input-1065870498.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)


--- Calculating indicators for TSLA ---
VIX data merged and VIX Regime calculated for TSLA.
Indicators calculated for TSLA. DataFrame shape: (150, 50)

        Data Fetching and Indicator Calculation Complete 

        Saving Analyzed Data to analyzed_financial_data.xlsx         
Analyzed data for ETF saved to sheet 'ETF'
Analyzed data for STOCK saved to sheet 'STOCK'

        Analyzed Data Export Complete                    


In [2]:
import pandas as pd
from datetime import datetime

# Assuming analyzed_data dictionary is available from the previous cell execution
if 'analyzed_data' in locals() or 'analyzed_data' in globals():
    # --- Add Timestamp to filename (Date only) ---
    now = datetime.now()
    timestamp = now.strftime("%d_%m_%Y") # Changed format to DD_MM_YYYY
    output_filename = f"analyzed_financial_data_{timestamp}.xlsx"
    # --- End Add Timestamp ---


    with pd.ExcelWriter(output_filename) as writer:
        for group_name, group_data in analyzed_data.items():
            if group_data: # Check if there is data for the group
                # Combine all tickers within the group into a single DataFrame for saving
                # Add a 'Ticker' column to identify the data for each ticker
                combined_df = pd.concat(
                    [df.assign(Ticker=ticker) for ticker, df in group_data.items()]
                )

                # Reset index so 'Date' becomes a regular column
                combined_df = combined_df.reset_index()

                # --- Reorder columns to place 'Ticker' after the 'Date' column ---
                # Get the list of columns, ensure 'Date' and 'Ticker' are present
                cols = combined_df.columns.tolist()
                if 'Date' in cols and 'Ticker' in cols:
                    # Remove 'Ticker' from its current position
                    cols.remove('Ticker')
                    # Find the index of the 'Date' column and insert 'Ticker' after it
                    date_index = cols.index('Date')
                    cols.insert(date_index + 1, 'Ticker')

                    # Reindex the DataFrame with the new column order
                    combined_df = combined_df[cols]
                # --- End Reorder columns ---


                # Save the combined DataFrame for the group to a sheet named after the group
                combined_df.to_excel(writer, sheet_name=group_name, index=False)

                print(f"Analyzed data for {group_name} saved to sheet '{group_name}' in {output_filename}")
            else:
                print(f"No analyzed data to save for {group_name} group.")

    print("\n=========================================================")
    print("        Analyzed Data Export Complete                    ")
    print(f"        Data saved to {output_filename}                 ")
    print("=========================================================")

else:
    print("Error: 'analyzed_data' variable not found. Please run the previous cell to analyze the data.")

Analyzed data for ETF saved to sheet 'ETF' in analyzed_financial_data_30_09_2025.xlsx
Analyzed data for STOCK saved to sheet 'STOCK' in analyzed_financial_data_30_09_2025.xlsx

        Analyzed Data Export Complete                    
        Data saved to analyzed_financial_data_30_09_2025.xlsx                 


In [3]:
!pip install pandas_ta
import pandas_ta as ta
import pandas as pd

# Assuming fetched_data is available from previous execution
if 'fetched_data' in locals() or 'fetched_data' in globals():
    analyzed_data = {}

    print("=========================================================")
    print("        Calculating Technical Indicators                 ")
    print("=========================================================")

    for group_name, group_data in fetched_data.items():
        print(f"\n################## Group: {group_name} ##################")
        analyzed_data[group_name] = {}

        for ticker, df in group_data.items():
            if not df.empty:
                print(f"\n--- Calculating indicators for {ticker} ---")
                # Calculate RSI (14)
                df.ta.rsi(length=14, append=True)

                # Calculate ATR (14)
                df.ta.atr(length=14, append=True)

                # Calculate Moving Averages
                df.ta.sma(length=50, append=True)
                df.ta.sma(length=100, append=True)
                df.ta.sma(length=150, append=True)
                df.ta.sma(length=200, append=True)

                # Calculate MACD (12, 26, 9)
                df.ta.macd(fast=12, slow=26, signal=9, append=True)

                analyzed_data[group_name][ticker] = df
                print(f"Indicators calculated for {ticker}. DataFrame shape: {df.shape}")
            else:
                print(f"!!! Skipping indicators for {ticker} as no data was available.")

    print("\n=========================================================")
    print("        Technical Indicator Calculation Complete         ")
    print("=========================================================")

    # You can now access the DataFrames with indicators in the 'analyzed_data' dictionary
    # For example, to see the data for SPY with new columns:
    # display(analyzed_data['ETF']['SPY'].tail())

else:
    print("Error: 'fetched_data' variable not found. Please run the previous cell to fetch the data.")

        Calculating Technical Indicators                 

################## Group: ETF ##################

--- Calculating indicators for SPY ---
Indicators calculated for SPY. DataFrame shape: (150, 13)

--- Calculating indicators for QQQ ---
Indicators calculated for QQQ. DataFrame shape: (150, 13)

--- Calculating indicators for MAGS ---
Indicators calculated for MAGS. DataFrame shape: (150, 13)

--- Calculating indicators for IBIT ---
Indicators calculated for IBIT. DataFrame shape: (150, 13)

--- Calculating indicators for ETHA ---
Indicators calculated for ETHA. DataFrame shape: (150, 13)

################## Group: STOCK ##################

--- Calculating indicators for AAPL ---
Indicators calculated for AAPL. DataFrame shape: (150, 13)

--- Calculating indicators for MSFT ---
Indicators calculated for MSFT. DataFrame shape: (150, 13)

--- Calculating indicators for GOOGL ---
Indicators calculated for GOOGL. DataFrame shape: (150, 13)

--- Calculating indicators for META ---
I

In [4]:
import yfinance as yf
import pandas as pd

# --- 1. שלב ראשון: הגדרת הנכסים ---

ASSET_GROUPS = {
    # נכסי ETF
    "ETF": ['SPY', 'QQQ', 'MAGS', 'IBIT', 'ETHA'],
    # מניות
    "STOCK": ['AAPL', 'MSFT', 'GOOGL', 'META', 'AMZN', 'NVDA', 'TSLA']
}

# --- 2. שלב שני: בחירת ימי המסחר (אינטראקטיבי מתוך הרשימה) ---

def select_trading_days():
    """
    מאפשר למשתמש לבחור את מספר ימי המסחר הרצוי מתוך הרשימה המוגדרת בלבד.
    In Colab, this will default to the maximum available days.
    """
    # ימי המסחר המוגדרים כפי שצוין במסמך
    AVAILABLE_DAYS = [50, 100, 150, 200, 300]
    DEFAULT_DAYS = AVAILABLE_DAYS[-1] # ברירת מחדל: 300 ימים

    print("\n=========================================================")
    print("  שלב שני: בחירת ימי מסחר אחרונים (היסטוריה)")
    print(f"  אפשרויות זמינות: {', '.join(map(str, AVAILABLE_DAYS))}")
    print("---------------------------------------------------------")
    print(f"  בסביבת Colab, נבחר אוטומטית ברירת מחדל: {DEFAULT_DAYS} ימים.")

    selected_days = DEFAULT_DAYS

    # Removed interactive input for Colab compatibility
    # try:
    #     user_input = input("אנא רשום את המספר הרצוי: ").strip()
    # except EOFError:
    #     user_input = ""
    #     print(f"(נבחר {DEFAULT_DAYS} ימים כברירת מחדל עקב סביבת הרצה.)")

    # if user_input != "":
    #     try:
    #         input_days = int(user_input)
    #         if input_days in AVAILABLE_DAYS:
    #             selected_days = input_days
    #         else:
    #             print(f"❌ המספר {input_days} אינו ברשימת האפשרויות. נבחר {DEFAULT_DAYS} ימים.")
    #     except ValueError:
    #         print(f"❌ קלט לא חוקי. נבחר {DEFAULT_DAYS} ימים.")

    print(f"✅ נבחרו {selected_days} ימי מסחר.")
    return selected_days

# --- 3. שלב שלישי: משיכת הנתונים הנדרשים (Open, Low, High, Close, Volume) ---

def get_base_data(ticker, days):
    """
    מושך נתונים היסטוריים (Open, Low, High, Close, Volume) עבור טיקר ומספר ימים נבחר.
    """
    try:
        # משיכה של שנתיים כדי להבטיח שיש 300 ימי מסחר
        df = yf.download(ticker, period="2y", progress=False)
        if df.empty:
            return pd.DataFrame()
    except Exception:
        return pd.DataFrame()

    # שמירת העמודות הנדרשות בלבד
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df_final = df[['Open', 'Low', 'High', 'Close', 'Volume']].tail(days)

    return df_final


# --- 4. לולאת ביצוע מרכזית ---

def run_stages_1_to_3():

    # שלב 2: בחירת ימי המסחר
    N_DAYS_SELECTED = select_trading_days()

    print("=========================================================")
    print(f"  יישום שלבים 1 ו-3: משיכת נתוני בסיס ({N_DAYS_SELECTED} ימים)")
    print("=========================================================")

    final_results = {} # Dictionary to store results

    for group_name, tickers in ASSET_GROUPS.items():
        print(f"\n################## טאב: {group_name} ##################")

        group_data = {} # Dictionary to store data for the current group
        for ticker in tickers:
            print(f"\n--- משיכת נתונים עבור {ticker} ---")

            # שלב 3: ביצוע משיכת הנתונים
            df_base = get_base_data(ticker, N_DAYS_SELECTED)

            if not df_base.empty:
                print(f"נתוני בסיס אחרונים (5 שורות) עבור {ticker}:")
                print(f"  (אורך הנתונים בפועל: {len(df_base)} ימים)")
                print(df_base.tail(5).to_markdown(floatfmt=".2f"))
                group_data[ticker] = df_base # Store the DataFrame in the group dictionary
            else:
                print(f"!!! דילוג: לא נמצאו נתונים עבור {ticker}.")

        final_results[group_name] = group_data # Store the group dictionary in the final results

    print("\n=========================================================")
    print("                 סיום יישום שלבים 1-3                     ")
    print("=========================================================")
    return final_results # Return the final results dictionary

# הרצת התוכנית
fetched_data = run_stages_1_to_3()


  שלב שני: בחירת ימי מסחר אחרונים (היסטוריה)
  אפשרויות זמינות: 50, 100, 150, 200, 300
---------------------------------------------------------
  בסביבת Colab, נבחר אוטומטית ברירת מחדל: 300 ימים.
✅ נבחרו 300 ימי מסחר.
  יישום שלבים 1 ו-3: משיכת נתוני בסיס (300 ימים)

################## טאב: ETF ##################

--- משיכת נתונים עבור SPY ---


/tmp/ipython-input-3888966216.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="2y", progress=False)


נתוני בסיס אחרונים (5 שורות) עבור SPY:
  (אורך הנתונים בפועל: 300 ימים)
| Date                |   Open |    Low |   High |   Close |      Volume |
|:--------------------|-------:|-------:|-------:|--------:|------------:|
| 2025-09-23 00:00:00 | 663.21 | 661.98 | 667.34 |  666.72 | 81708900.00 |
| 2025-09-24 00:00:00 | 661.10 | 659.67 | 664.61 |  664.51 | 68082200.00 |
| 2025-09-25 00:00:00 | 658.05 | 654.41 | 659.41 |  657.94 | 89622100.00 |
| 2025-09-26 00:00:00 | 661.82 | 657.88 | 662.37 |  659.51 | 69179200.00 |
| 2025-09-29 00:00:00 | 663.68 | 661.86 | 665.28 |  664.36 | 73415500.00 |

--- משיכת נתונים עבור QQQ ---


/tmp/ipython-input-3888966216.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="2y", progress=False)
/tmp/ipython-input-3888966216.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="2y", progress=False)


נתוני בסיס אחרונים (5 שורות) עבור QQQ:
  (אורך הנתונים בפועל: 300 ימים)
| Date                |   Open |    Low |   High |   Close |      Volume |
|:--------------------|-------:|-------:|-------:|--------:|------------:|
| 2025-09-23 00:00:00 | 598.20 | 596.98 | 602.57 |  602.37 | 64635500.00 |
| 2025-09-24 00:00:00 | 596.10 | 593.36 | 599.90 |  599.58 | 49850300.00 |
| 2025-09-25 00:00:00 | 593.53 | 588.50 | 595.12 |  592.20 | 70920200.00 |
| 2025-09-26 00:00:00 | 595.97 | 591.06 | 596.30 |  594.35 | 54337400.00 |
| 2025-09-29 00:00:00 | 598.73 | 597.41 | 602.05 |  599.11 | 48255400.00 |

--- משיכת נתונים עבור MAGS ---
נתוני בסיס אחרונים (5 שורות) עבור MAGS:
  (אורך הנתונים בפועל: 300 ימים)
| Date                |   Open |   Low |   High |   Close |     Volume |
|:--------------------|-------:|------:|-------:|--------:|-----------:|
| 2025-09-23 00:00:00 |  64.57 | 64.44 |  65.73 |   65.73 | 2660400.00 |
| 2025-09-24 00:00:00 |  64.64 | 64.17 |  64.99 |   64.93 | 1859600.00 |
| 2025

...

/tmp/ipython-input-3888966216.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="2y", progress=False)


נתוני בסיס אחרונים (5 שורות) עבור META:
  (אורך הנתונים בפועל: 300 ימים)
| Date                |   Open |    Low |   High |   Close |      Volume |
|:--------------------|-------:|-------:|-------:|--------:|------------:|
| 2025-09-23 00:00:00 | 755.40 | 751.07 | 770.60 |  769.25 | 10872600.00 |
| 2025-09-24 00:00:00 | 760.66 | 752.53 | 761.11 |  757.50 |  8828200.00 |
| 2025-09-25 00:00:00 | 748.91 | 744.55 | 756.77 |  753.45 | 10591100.00 |
| 2025-09-26 00:00:00 | 743.75 | 737.35 | 751.93 |  750.00 |  9696300.00 |
| 2025-09-29 00:00:00 | 743.40 | 739.15 | 750.78 |  748.72 |  9228700.00 |

--- משיכת נתונים עבור AMZN ---
נתוני בסיס אחרונים (5 שורות) עבור AMZN:
  (אורך הנתונים בפועל: 300 ימים)
| Date                |   Open |    Low |   High |   Close |      Volume |
|:--------------------|-------:|-------:|-------:|--------:|------------:|
| 2025-09-23 00:00:00 | 220.71 | 220.07 | 227.86 |  227.83 | 70956200.00 |
| 2025-09-24 00:00:00 | 220.21 | 219.45 | 224.56 |  224.15 | 49509000.00

/tmp/ipython-input-3888966216.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="2y", progress=False)
/tmp/ipython-input-3888966216.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="2y", progress=False)


נתוני בסיס אחרונים (5 שורות) עבור NVDA:
  (אורך הנתונים בפועל: 300 ימים)
| Date                |   Open |    Low |   High |   Close |       Volume |
|:--------------------|-------:|-------:|-------:|--------:|-------------:|
| 2025-09-23 00:00:00 | 178.43 | 176.21 | 182.42 |  181.97 | 192559600.00 |
| 2025-09-24 00:00:00 | 176.97 | 175.40 | 179.78 |  179.77 | 143564100.00 |
| 2025-09-25 00:00:00 | 177.69 | 173.13 | 180.26 |  174.48 | 191586700.00 |
| 2025-09-26 00:00:00 | 178.19 | 174.93 | 179.77 |  178.17 | 148573700.00 |
| 2025-09-29 00:00:00 | 181.85 | 180.32 | 184.00 |  180.43 | 192662800.00 |

--- משיכת נתונים עבור TSLA ---
נתוני בסיס אחרונים (5 שורות) עבור TSLA:
  (אורך הנתונים בפועל: 300 ימים)
| Date                |   Open |    Low |   High |   Close |       Volume |
|:--------------------|-------:|-------:|-------:|--------:|-------------:|
| 2025-09-23 00:00:00 | 425.85 | 423.72 | 440.97 |  439.88 |  83422700.00 |
| 2025-09-24 00:00:00 | 442.79 | 429.03 | 444.21 |  429.83 |  

/tmp/ipython-input-3888966216.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="2y", progress=False)


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
from datetime import datetime

# Assuming fetched_data dictionary is available from the previous cell execution
if 'fetched_data' in locals() or 'fetched_data' in globals():
    # --- Get Date Timestamp ---
    now = datetime.now()
    timestamp = now.strftime("%d_%m_%Y") # Changed format to DD_MM_YYYY
    # --- End Get Date Timestamp ---

    for group_name, group_data in fetched_data.items():
        if group_data: # Check if there is data for the group
            # Combine all tickers within the group into a single DataFrame for saving
            # Add a 'Ticker' column to identify the data for each ticker
            combined_df = pd.concat(
                [df.assign(Ticker=ticker) for ticker, df in group_data.items()]
            )

            # Reset index so 'Date' becomes a regular column
            combined_df = combined_df.reset_index()

            # --- Reorder columns to place 'Ticker' after the 'Date' column ---
            # Get the list of columns, ensure 'Date' and 'Ticker' are present
            cols = combined_df.columns.tolist()
            if 'Date' in cols and 'Ticker' in cols:
                # Remove 'Ticker' from its current position
                cols.remove('Ticker')
                # Find the index of the 'Date' column and insert 'Ticker' after it
                date_index = cols.index('Date')
                cols.insert(date_index + 1, 'Ticker')

                # Reindex the DataFrame with the new column order
                combined_df = combined_df[cols]
            # --- End Reorder columns ---

            # Define the filename based on the group name and timestamp
            filename = f"{group_name}_{timestamp}.csv" # Added timestamp to filename

            # Save the combined DataFrame to a CSV file
            # Reset index so 'Date' becomes a regular column for easier reordering
            combined_df.reset_index().to_csv(filename, index=False)


            print(f"Full results for {group_name} saved to {filename}")
        else:
            print(f"No data to save for {group_name} group.")

else:
    print("Error: 'fetched_data' variable not found. Please run the previous cell to generate the results.")

Full results for ETF saved to ETF_30_09_2025.csv
Full results for STOCK saved to STOCK_30_09_2025.csv


# ****** Base App *v2* *****
Add the VIX and VIX Regime indicators to the analysis.

## Fetch vix data

### Subtask:
Add code to cell `fe783563` to fetch historical data for the VIX ticker (`^VIX`) using `yfinance` for the same time range (`N_DAYS_SELECTED`) as the other assets.


**Reasoning**:
I need to add a new function `get_vix_data` to cell `fe783563` that fetches VIX data and then call this function before the inner loop processing tickers.



In [7]:
# Install necessary libraries
!pip install pandas_ta
import yfinance as yf
import pandas as pd
import pandas_ta as ta
from datetime import datetime # Import datetime for getting current time

# --- 1. Define Assets ---
ASSET_GROUPS = {
    "ETF": ['SPY', 'QQQ', 'MAGS', 'IBIT', 'ETHA'],
    "STOCK": ['AAPL', 'MSFT', 'GOOGL', 'META', 'AMZN', 'NVDA', 'TSLA']
}

# --- 2. Define Trading Days (Adjusted for Short/Medium Term) ---
N_DAYS_SELECTED = 150 # Focusing on short to medium term trends

# --- 3. Function to Fetch Base Data ---
def get_base_data(ticker, days):
    """
    Fetches historical data (Open, Low, High, Close, Volume) for a given ticker.
    """
    try:
        # Fetch enough data to ensure the selected number of days are available
        # Fetching 1 year should be sufficient for 150 days
        df = yf.download(ticker, period="1y", progress=False)
        if df.empty:
            return pd.DataFrame()
    except Exception:
        return pd.DataFrame()

    # Keep only the required columns
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df_final = df[['Open', 'Low', 'High', 'Close', 'Volume']].tail(days)

    return df_final

# --- Function to Fetch Current Price (for Crypto) ---
def get_current_price(ticker):
    """
    Fetches the current price for a given ticker.
    Uses yfinance for consistency, may return delayed data.
    For true real-time, a crypto-specific API would be needed.
    """
    try:
        # Fetch a very short period to get the latest data point
        ticker_info = yf.Ticker(ticker)
        # Get the most recent price data
        latest_data = ticker_info.history(period="1d", interval="1m")
        if not latest_data.empty:
            return latest_data['Close'].iloc[-1]
        else:
            # Fallback: try fetching info
             info = ticker_info.info
             if 'currentPrice' in info:
                 return info['currentPrice']
             return None
    except Exception:
        return None

# --- Function to Fetch VIX Data ---
def get_vix_data(days):
    """
    Fetches historical Close data for the VIX (^VIX) ticker.
    """
    try:
        df = yf.download('^VIX', period="1y", progress=False)
        if df.empty:
            return pd.DataFrame()
    except Exception:
        return pd.DataFrame()

    # Keep only the 'Close' column and rename it
    df_vix = df[['Close']].copy()
    df_vix.columns = ['VIX_Close']

    # Return the last 'days' rows
    return df_vix.tail(days)


# --- 4. Fetch Data and Calculate Indicators ---
fetched_data = {}
analyzed_data = {}

print("=========================================================")
print(f"        Fetching Data and Calculating Indicators ({N_DAYS_SELECTED} days)")
print("=========================================================")

# Fetch VIX data once
df_vix = get_vix_data(N_DAYS_SELECTED)
if df_vix.empty:
    print("!!! Could not fetch VIX data. VIX-related indicators will be skipped.")
else:
    print(f"VIX data fetched. DataFrame shape: {df_vix.shape}")


for group_name, tickers in ASSET_GROUPS.items():
    print(f"\n################## Group: {group_name} ##################")
    fetched_data[group_name] = {}
    analyzed_data[group_name] = {}

    for ticker in tickers:
        print(f"\n--- Processing data for {ticker} ---")

        # Fetch data
        df_base = get_base_data(ticker, N_DAYS_SELECTED)

        if not df_base.empty:
            fetched_data[group_name][ticker] = df_base.copy() # Store fetched data

            # Calculate technical indicators using pandas_ta (Relevant for short/medium term)
            print(f"--- Calculating indicators for {ticker} ---")
            df_analysis = df_base.copy() # Work on a copy
            df_analysis.ta.rsi(length=14, append=True)
            df_analysis.ta.atr(length=14, append=True)
            df_analysis.ta.sma(length=50, append=True) # Medium-term MA
            df_analysis.ta.sma(length=100, append=True) # Longer end of medium-term MA
            df_analysis.ta.macd(fast=12, slow=26, signal=9, append=True)

            # Add more indicators (examples)
            df_analysis.ta.bbands(length=20, append=True) # Bollinger Bands
            df_analysis.ta.ema(length=20, append=True) # Exponential Moving Average (Short-term)
            df_analysis.ta.vwap(append=True) # Volume Weighted Average Price

            # Add Stochastic Oscillator with specified parameters (Short-term)
            stoch_k, stoch_d, stoch_smooth_k = 14, 3, 3
            df_analysis.ta.stoch(k=stoch_k, d=stoch_d, smooth_k=stoch_smooth_k, append=True)

            # --- Add Logic to Combine Stochastic with MAs (Adjusted for Medium Term) ---
            # Example: Check if Stochastic %K is above SMA 50 (Potential bullish signal relative to medium trend)
            stoch_k_col = f'STOCHk_{stoch_k}_{stoch_d}_{stoch_smooth_k}'
            sma_50_col = 'SMA_50'
            if stoch_k_col in df_analysis.columns and sma_50_col in df_analysis.columns:
                df_analysis['STOCHk_vs_SMA50_Above'] = (df_analysis[stoch_k_col] > df_analysis[sma_50_col]).astype(int)

            # Removed comparison to SMA 200 as we are focusing on shorter/medium terms

            # --- Add Fibonacci Retracement and Extension Levels (Based on Daily Data) ---
            # Calculate based on the High and Low of the current daily data range (150 days)
            fib_max_price_daily = df_analysis['High'].max()
            fib_min_price_daily = df_analysis['Low'].min()
            fib_range_daily = fib_max_price_daily - fib_min_price_daily

            # Fibonacci Retracement Levels (Daily)
            df_analysis['Fib_0_Retracement_Daily'] = fib_max_price_daily
            df_analysis['Fib_23.6_Retracement_Daily'] = fib_max_price_daily - (fib_range_daily * 0.236)
            df_analysis['Fib_38.2_Retracement_Daily'] = fib_max_price_daily - (fib_range_daily * 0.382)
            df_analysis['Fib_50.0_Retracement_Daily'] = fib_range_daily * 0.50 + fib_min_price_daily # Corrected 50% retracement calculation
            df_analysis['Fib_61.8_Retracement_Daily'] = fib_max_price_daily - (fib_range_daily * 0.618)
            df_analysis['Fib_78.6_Retracement_Daily'] = fib_max_price_daily - (fib_range_daily * 0.786)
            df_analysis['Fib_100_Retracement_Daily'] = fib_min_price_daily

            # Fibonacci Extension Levels (Daily)
            df_analysis['Fib_123.6_Extension_Daily'] = fib_max_price_daily + (fib_range_daily * 0.236)
            df_analysis['Fib_138.2_Extension_Daily'] = fib_max_price_daily + (fib_range_daily * 0.382)
            df_analysis['Fib_161.8_Extension_Daily'] = fib_max_price_daily + (fib_range_daily * 0.618)
            df_analysis['Fib_200.0_Extension_Daily'] = fib_max_price_daily + (fib_range_daily * 1.00)
            df_analysis['Fib_261.8_Extension_Daily'] = fib_max_price_daily + (fib_range_daily * 1.618)


            # --- Add Fibonacci Retracement and Extension Levels (Based on Weekly Data) ---
            # Resample daily data to weekly
            df_weekly = df_base.resample('W').agg({
                'Open': 'first',
                'High': 'max',
                'Low': 'min',
                'Close': 'last',
                'Volume': 'sum'
            }).dropna() # Drop weeks with no data

            if not df_weekly.empty:
                # Calculate based on the High and Low of the current weekly data range
                fib_max_price_weekly = df_weekly['High'].max()
                fib_min_price_weekly = df_weekly['Low'].min()
                fib_range_weekly = fib_max_price_weekly - fib_min_price_weekly

                # Fibonacci Retracement Levels (Weekly)
                df_analysis['Fib_0_Retracement_Weekly'] = fib_max_price_weekly
                df_analysis['Fib_23.6_Retracement_Weekly'] = fib_max_price_weekly - (fib_range_weekly * 0.236)
                df_analysis['Fib_38.2_Retracement_Weekly'] = fib_max_price_weekly - (fib_range_weekly * 0.382)
                df_analysis['Fib_50.0_Retracement_Weekly'] = fib_range_weekly * 0.50 + fib_min_price_weekly # Corrected 50% retracement calculation
                df_analysis['Fib_61.8_Retracement_Weekly'] = fib_max_price_weekly - (fib_range_weekly * 0.618)
                df_analysis['Fib_78.6_Retracement_Weekly'] = fib_max_price_weekly - (fib_range_weekly * 0.786)
                df_analysis['Fib_100_Retracement_Weekly'] = fib_min_price_weekly

                # Fibonacci Extension Levels (Weekly)
                df_analysis['Fib_123.6_Extension_Weekly'] = fib_max_price_weekly + (fib_range_weekly * 0.236)
                df_analysis['Fib_138.2_Extension_Weekly'] = fib_max_price_weekly + (fib_range_weekly * 0.382)
                df_analysis['Fib_161.8_Extension_Weekly'] = fib_max_price_weekly + (fib_range_weekly * 0.618)
                df_analysis['Fib_200.0_Extension_Weekly'] = fib_max_price_weekly + (fib_range_weekly * 1.00)
                df_analysis['Fib_261.8_Extension_Weekly'] = fib_max_price_weekly + (fib_range_weekly * 1.618)
            else:
                 print(f"!!! No sufficient weekly data for {ticker} to calculate Weekly Fibonacci levels.")

            # --- Fetch and Add Current Crypto Price ---
            if ticker == 'IBIT':
                btc_ticker = 'BTC-USD'
                current_btc_price = get_current_price(btc_ticker)
                if current_btc_price is not None:
                    # Add current price to the last row of the IBIT DataFrame
                    # Create new columns for current price and its timestamp
                    current_time = datetime.now()
                    df_analysis.loc[df_analysis.index[-1], f'Current_Price_{btc_ticker}'] = current_btc_price
                    df_analysis.loc[df_analysis.index[-1], f'Current_Price_Timestamp_{btc_ticker}'] = current_time
                    print(f"Added current price for {btc_ticker}: {current_btc_price}")
                else:
                    print(f"!!! Could not fetch current price for {btc_ticker}.")

            elif ticker == 'ETHA':
                eth_ticker = 'ETH-USD'
                current_eth_price = get_current_price(eth_ticker)
                if current_eth_price is not None:
                     # Add current price to the last row of the ETHA DataFrame
                    # Create new columns for current price and its timestamp
                    current_time = datetime.now()
                    df_analysis.loc[df_analysis.index[-1], f'Current_Price_{eth_ticker}'] = current_eth_price
                    df_analysis.loc[df_analysis.index[-1], f'Current_Price_Timestamp_{eth_ticker}'] = current_time
                    print(f"Added current price for {eth_ticker}: {current_eth_price}")
                else:
                    print(f"!!! Could not fetch current price for {eth_ticker}.")
            # --- End Fetch and Add Current Crypto Price ---


            analyzed_data[group_name][ticker] = df_analysis # Store analyzed data
            print(f"Indicators calculated for {ticker}. DataFrame shape: {df_analysis.shape}")
        else:
            print(f"!!! Skipping {ticker} as no data was available.")

print("\n=========================================================")
print("        Data Fetching and Indicator Calculation Complete ")
print("=========================================================")

# --- 5. Save Analyzed Data to Excel ---
output_filename = "analyzed_financial_data.xlsx"

print(f"\n=========================================================")
print(f"        Saving Analyzed Data to {output_filename}         ")
print("=========================================================")

with pd.ExcelWriter(output_filename) as writer:
    for group_name, group_data in analyzed_data.items():
        if group_data: # Check if there is data for the group
            # Combine all tickers within the group into a single DataFrame for saving
            # Add a 'Ticker' column to identify the data for each ticker
            combined_df = pd.concat(
                [df.assign(Ticker=ticker) for ticker, df in group_data.items()]
            )

            # Save the combined DataFrame for the group to a sheet named after the group
            combined_df.to_excel(writer, sheet_name=group_name)

            print(f"Analyzed data for {group_name} saved to sheet '{group_name}'")
        else:
            print(f"No analyzed data to save for {group_name} group.")

print("\n=========================================================")
print("        Analyzed Data Export Complete                    ")
print("=========================================================")

        Fetching Data and Calculating Indicators (150 days)
VIX data fetched. DataFrame shape: (150, 1)

################## Group: ETF ##################

--- Processing data for SPY ---


/tmp/ipython-input-3310329532.py:66: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('^VIX', period="1y", progress=False)
/tmp/ipython-input-3310329532.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)


--- Calculating indicators for SPY ---
Indicators calculated for SPY. DataFrame shape: (150, 47)

--- Processing data for QQQ ---


/tmp/ipython-input-3310329532.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)


--- Calculating indicators for QQQ ---
Indicators calculated for QQQ. DataFrame shape: (150, 47)

--- Processing data for MAGS ---
--- Calculating indicators for MAGS ---


...

--- Calculating indicators for GOOGL ---
Indicators calculated for GOOGL. DataFrame shape: (150, 47)

--- Processing data for META ---
--- Calculating indicators for META ---
Indicators calculated for META. DataFrame shape: (150, 47)

--- Processing data for AMZN ---


/tmp/ipython-input-3310329532.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)
/tmp/ipython-input-3310329532.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)


--- Calculating indicators for AMZN ---
Indicators calculated for AMZN. DataFrame shape: (150, 47)

--- Processing data for NVDA ---
--- Calculating indicators for NVDA ---
Indicators calculated for NVDA. DataFrame shape: (150, 47)

--- Processing data for TSLA ---


/tmp/ipython-input-3310329532.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)
/tmp/ipython-input-3310329532.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)


--- Calculating indicators for TSLA ---
Indicators calculated for TSLA. DataFrame shape: (150, 47)

        Data Fetching and Indicator Calculation Complete 

        Saving Analyzed Data to analyzed_financial_data.xlsx         
Analyzed data for ETF saved to sheet 'ETF'
Analyzed data for STOCK saved to sheet 'STOCK'

        Analyzed Data Export Complete                    


**Reasoning**:
I have successfully added the `get_vix_data` function and fetched the VIX data. Now I need to merge the VIX data with the asset dataframes and calculate the VIX Regime indicator within the loop that processes each ticker.



In [8]:
# Install necessary libraries
!pip install pandas_ta
import yfinance as yf
import pandas as pd
import pandas_ta as ta
from datetime import datetime # Import datetime for getting current time

# --- 1. Define Assets ---
ASSET_GROUPS = {
    "ETF": ['SPY', 'QQQ', 'MAGS', 'IBIT', 'ETHA'],
    "STOCK": ['AAPL', 'MSFT', 'GOOGL', 'META', 'AMZN', 'NVDA', 'TSLA']
}

# --- 2. Define Trading Days (Adjusted for Short/Medium Term) ---
N_DAYS_SELECTED = 150 # Focusing on short to medium term trends

# --- 3. Function to Fetch Base Data ---
def get_base_data(ticker, days):
    """
    Fetches historical data (Open, Low, High, Close, Volume) for a given ticker.
    """
    try:
        # Fetch enough data to ensure the selected number of days are available
        # Fetching 1 year should be sufficient for 150 days
        df = yf.download(ticker, period="1y", progress=False)
        if df.empty:
            return pd.DataFrame()
    except Exception:
        return pd.DataFrame()

    # Keep only the required columns
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df_final = df[['Open', 'Low', 'High', 'Close', 'Volume']].tail(days)

    return df_final

# --- Function to Fetch Current Price (for Crypto) ---
def get_current_price(ticker):
    """
    Fetches the current price for a given ticker.
    Uses yfinance for consistency, may return delayed data.
    For true real-time, a crypto-specific API would be needed.
    """
    try:
        # Fetch a very short period to get the latest data point
        ticker_info = yf.Ticker(ticker)
        # Get the most recent price data
        latest_data = ticker_info.history(period="1d", interval="1m")
        if not latest_data.empty:
            return latest_data['Close'].iloc[-1]
        else:
            # Fallback: try fetching info
             info = ticker_info.info
             if 'currentPrice' in info:
                 return info['currentPrice']
             return None
    except Exception:
        return None

# --- Function to Fetch VIX Data ---
def get_vix_data(days):
    """
    Fetches historical Close data for the VIX (^VIX) ticker.
    """
    try:
        df = yf.download('^VIX', period="1y", progress=False)
        if df.empty:
            return pd.DataFrame()
    except Exception:
        return pd.DataFrame()

    # Keep only the 'Close' column and rename it
    df_vix = df[['Close']].copy()
    df_vix.columns = ['VIX_Close']

    # Return the last 'days' rows
    return df_vix.tail(days)


# --- 4. Fetch Data and Calculate Indicators ---
fetched_data = {}
analyzed_data = {}

print("=========================================================")
print(f"        Fetching Data and Calculating Indicators ({N_DAYS_SELECTED} days)")
print("=========================================================")

# Fetch VIX data once
df_vix = get_vix_data(N_DAYS_SELECTED)
if df_vix.empty:
    print("!!! Could not fetch VIX data. VIX-related indicators will be skipped.")
else:
    print(f"VIX data fetched. DataFrame shape: {df_vix.shape}")


for group_name, tickers in ASSET_GROUPS.items():
    print(f"\n################## Group: {group_name} ##################")
    fetched_data[group_name] = {}
    analyzed_data[group_name] = {}

    for ticker in tickers:
        print(f"\n--- Processing data for {ticker} ---")

        # Fetch data
        df_base = get_base_data(ticker, N_DAYS_SELECTED)

        if not df_base.empty:
            fetched_data[group_name][ticker] = df_base.copy() # Store fetched data

            # Calculate technical indicators using pandas_ta (Relevant for short/medium term)
            print(f"--- Calculating indicators for {ticker} ---")
            df_analysis = df_base.copy() # Work on a copy
            df_analysis.ta.rsi(length=14, append=True)
            df_analysis.ta.atr(length=14, append=True)
            df_analysis.ta.sma(length=50, append=True) # Medium-term MA
            df_analysis.ta.sma(length=100, append=True) # Longer end of medium-term MA
            df_analysis.ta.macd(fast=12, slow=26, signal=9, append=True)

            # Add more indicators (examples)
            df_analysis.ta.bbands(length=20, append=True) # Bollinger Bands
            df_analysis.ta.ema(length=20, append=True) # Exponential Moving Average (Short-term)
            df_analysis.ta.vwap(append=True) # Volume Weighted Average Price

            # Add Stochastic Oscillator with specified parameters (Short-term)
            stoch_k, stoch_d, stoch_smooth_k = 14, 3, 3
            df_analysis.ta.stoch(k=stoch_k, d=stoch_d, smooth_k=stoch_smooth_k, append=True)

            # --- Add Logic to Combine Stochastic with MAs (Adjusted for Medium Term) ---
            # Example: Check if Stochastic %K is above SMA 50 (Potential bullish signal relative to medium trend)
            stoch_k_col = f'STOCHk_{stoch_k}_{stoch_d}_{stoch_smooth_k}'
            sma_50_col = 'SMA_50'
            if stoch_k_col in df_analysis.columns and sma_50_col in df_analysis.columns:
                df_analysis['STOCHk_vs_SMA50_Above'] = (df_analysis[stoch_k_col] > df_analysis[sma_50_col]).astype(int)

            # Removed comparison to SMA 200 as we are focusing on shorter/medium terms

            # --- Add Fibonacci Retracement and Extension Levels (Based on Daily Data) ---
            # Calculate based on the High and Low of the current daily data range (150 days)
            fib_max_price_daily = df_analysis['High'].max()
            fib_min_price_daily = df_analysis['Low'].min()
            fib_range_daily = fib_max_price_daily - fib_min_price_daily

            # Fibonacci Retracement Levels (Daily)
            df_analysis['Fib_0_Retracement_Daily'] = fib_max_price_daily
            df_analysis['Fib_23.6_Retracement_Daily'] = fib_max_price_daily - (fib_range_daily * 0.236)
            df_analysis['Fib_38.2_Retracement_Daily'] = fib_max_price_daily - (fib_range_daily * 0.382)
            df_analysis['Fib_50.0_Retracement_Daily'] = fib_range_daily * 0.50 + fib_min_price_daily # Corrected 50% retracement calculation
            df_analysis['Fib_61.8_Retracement_Daily'] = fib_max_price_daily - (fib_range_daily * 0.618)
            df_analysis['Fib_78.6_Retracement_Daily'] = fib_max_price_daily - (fib_range_daily * 0.786)
            df_analysis['Fib_100_Retracement_Daily'] = fib_min_price_daily

            # Fibonacci Extension Levels (Daily)
            df_analysis['Fib_123.6_Extension_Daily'] = fib_max_price_daily + (fib_range_daily * 0.236)
            df_analysis['Fib_138.2_Extension_Daily'] = fib_max_price_daily + (fib_range_daily * 0.382)
            df_analysis['Fib_161.8_Extension_Daily'] = fib_max_price_daily + (fib_range_daily * 0.618)
            df_analysis['Fib_200.0_Extension_Daily'] = fib_max_price_daily + (fib_range_daily * 1.00)
            df_analysis['Fib_261.8_Extension_Daily'] = fib_max_price_daily + (fib_range_daily * 1.618)


            # --- Add Fibonacci Retracement and Extension Levels (Based on Weekly Data) ---
            # Resample daily data to weekly
            df_weekly = df_base.resample('W').agg({
                'Open': 'first',
                'High': 'max',
                'Low': 'min',
                'Close': 'last',
                'Volume': 'sum'
            }).dropna() # Drop weeks with no data

            if not df_weekly.empty:
                # Calculate based on the High and Low of the current weekly data range
                fib_max_price_weekly = df_weekly['High'].max()
                fib_min_price_weekly = df_weekly['Low'].min()
                fib_range_weekly = fib_max_price_weekly - fib_min_price_weekly

                # Fibonacci Retracement Levels (Weekly)
                df_analysis['Fib_0_Retracement_Weekly'] = fib_max_price_weekly
                df_analysis['Fib_23.6_Retracement_Weekly'] = fib_max_price_weekly - (fib_range_weekly * 0.236)
                df_analysis['Fib_38.2_Retracement_Weekly'] = fib_max_price_weekly - (fib_range_weekly * 0.382)
                df_analysis['Fib_50.0_Retracement_Weekly'] = fib_range_weekly * 0.50 + fib_min_price_weekly # Corrected 50% retracement calculation
                df_analysis['Fib_61.8_Retracement_Weekly'] = fib_max_price_weekly - (fib_range_weekly * 0.618)
                df_analysis['Fib_78.6_Retracement_Weekly'] = fib_max_price_weekly - (fib_range_weekly * 0.786)
                df_analysis['Fib_100_Retracement_Weekly'] = fib_min_price_weekly

                # Fibonacci Extension Levels (Weekly)
                df_analysis['Fib_123.6_Extension_Weekly'] = fib_max_price_weekly + (fib_range_weekly * 0.236)
                df_analysis['Fib_138.2_Extension_Weekly'] = fib_max_price_weekly + (fib_range_weekly * 0.382)
                df_analysis['Fib_161.8_Extension_Weekly'] = fib_max_price_weekly + (fib_range_weekly * 0.618)
                df_analysis['Fib_200.0_Extension_Weekly'] = fib_max_price_weekly + (fib_range_weekly * 1.00)
                df_analysis['Fib_261.8_Extension_Weekly'] = fib_max_price_weekly + (fib_range_weekly * 1.618)
            else:
                 print(f"!!! No sufficient weekly data for {ticker} to calculate Weekly Fibonacci levels.")

            # --- Add VIX data and VIX Regime Indicator ---
            if not df_vix.empty:
                # Merge VIX data with the current asset's data
                df_analysis = pd.merge(df_analysis, df_vix, left_index=True, right_index=True, how='left')

                # Calculate VIX Regime (simple example: above/below a threshold)
                # You might define a more sophisticated regime based on moving averages, volatility bands, etc.
                vix_threshold = 20 # Example threshold
                df_analysis['VIX_Regime_Above_Threshold'] = (df_analysis['VIX_Close'] > vix_threshold).astype(int)
                print(f"VIX data merged and VIX Regime calculated for {ticker}.")
            else:
                 print(f"!!! Skipping VIX-related indicators for {ticker} due to missing VIX data.")
            # --- End Add VIX data and VIX Regime Indicator ---


            # --- Fetch and Add Current Crypto Price ---
            if ticker == 'IBIT':
                btc_ticker = 'BTC-USD'
                current_btc_price = get_current_price(btc_ticker)
                if current_btc_price is not None:
                    # Add current price to the last row of the IBIT DataFrame
                    # Create new columns for current price and its timestamp
                    current_time = datetime.now()
                    df_analysis.loc[df_analysis.index[-1], f'Current_Price_{btc_ticker}'] = current_btc_price
                    df_analysis.loc[df_analysis.index[-1], f'Current_Price_Timestamp_{btc_ticker}'] = current_time
                    print(f"Added current price for {btc_ticker}: {current_btc_price}")
                else:
                    print(f"!!! Could not fetch current price for {btc_ticker}.")

            elif ticker == 'ETHA':
                eth_ticker = 'ETH-USD'
                current_eth_price = get_current_price(eth_ticker)
                if current_eth_price is not None:
                     # Add current price to the last row of the ETHA DataFrame
                    # Create new columns for current price and its timestamp
                    current_time = datetime.now()
                    df_analysis.loc[df_analysis.index[-1], f'Current_Price_{eth_ticker}'] = current_eth_price
                    df_analysis.loc[df_analysis.index[-1], f'Current_Price_Timestamp_{eth_ticker}'] = current_time
                    print(f"Added current price for {eth_ticker}: {current_eth_price}")
                else:
                    print(f"!!! Could not fetch current price for {eth_ticker}.")
            # --- End Fetch and Add Current Crypto Price ---


            analyzed_data[group_name][ticker] = df_analysis # Store analyzed data
            print(f"Indicators calculated for {ticker}. DataFrame shape: {df_analysis.shape}")
        else:
            print(f"!!! Skipping {ticker} as no data was available.")

print("\n=========================================================")
print("        Data Fetching and Indicator Calculation Complete ")
print("=========================================================")

# --- 5. Save Analyzed Data to Excel ---
output_filename = "analyzed_financial_data.xlsx"

print(f"\n=========================================================")
print(f"        Saving Analyzed Data to {output_filename}         ")
print("=========================================================")

with pd.ExcelWriter(output_filename) as writer:
    for group_name, group_data in analyzed_data.items():
        if group_data: # Check if there is data for the group
            # Combine all tickers within the group into a single DataFrame for saving
            # Add a 'Ticker' column to identify the data for each ticker
            combined_df = pd.concat(
                [df.assign(Ticker=ticker) for ticker, df in group_data.items()]
            )

            # Save the combined DataFrame for the group to a sheet named after the group
            combined_df.to_excel(writer, sheet_name=group_name)

            print(f"Analyzed data for {group_name} saved to sheet '{group_name}'")
        else:
            print(f"No analyzed data to save for {group_name} group.")

print("\n=========================================================")
print("        Analyzed Data Export Complete                    ")
print("=========================================================")

        Fetching Data and Calculating Indicators (150 days)


/tmp/ipython-input-785959191.py:66: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('^VIX', period="1y", progress=False)


VIX data fetched. DataFrame shape: (150, 1)

################## Group: ETF ##################

--- Processing data for SPY ---


/tmp/ipython-input-785959191.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)


--- Calculating indicators for SPY ---
VIX data merged and VIX Regime calculated for SPY.
Indicators calculated for SPY. DataFrame shape: (150, 49)

--- Processing data for QQQ ---


...

--- Calculating indicators for GOOGL ---
VIX data merged and VIX Regime calculated for GOOGL.
Indicators calculated for GOOGL. DataFrame shape: (150, 49)

--- Processing data for META ---


/tmp/ipython-input-785959191.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)


--- Calculating indicators for META ---
VIX data merged and VIX Regime calculated for META.
Indicators calculated for META. DataFrame shape: (150, 49)

--- Processing data for AMZN ---
--- Calculating indicators for AMZN ---
VIX data merged and VIX Regime calculated for AMZN.
Indicators calculated for AMZN. DataFrame shape: (150, 49)

--- Processing data for NVDA ---


/tmp/ipython-input-785959191.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)
/tmp/ipython-input-785959191.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)
/tmp/ipython-input-785959191.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)


--- Calculating indicators for NVDA ---
VIX data merged and VIX Regime calculated for NVDA.
Indicators calculated for NVDA. DataFrame shape: (150, 49)

--- Processing data for TSLA ---
--- Calculating indicators for TSLA ---
VIX data merged and VIX Regime calculated for TSLA.
Indicators calculated for TSLA. DataFrame shape: (150, 49)

        Data Fetching and Indicator Calculation Complete 

        Saving Analyzed Data to analyzed_financial_data.xlsx         
Analyzed data for ETF saved to sheet 'ETF'
Analyzed data for STOCK saved to sheet 'STOCK'

        Analyzed Data Export Complete                    


## Define vix regime logic

### Subtask:
Define or ask the user for the thresholds that define "Low" and "High" VIX regimes.


## Review and refine

### Subtask:
Review the modified cell `fe783563` to ensure the VIX data and Regime are correctly integrated and the output DataFrames in `analyzed_data` contain the new columns.


**Reasoning**:
Review the code in cell fe783563 to ensure the VIX data and Regime are correctly integrated and the output DataFrames in analyzed_data contain the new columns, then finish the task.



In [9]:
# Examine the code in cell fe783563 (this cell).
# Verify that the get_vix_data function is correctly defined and called once before the main asset processing loop.
# Confirm that the merging of df_vix with df_analysis is performed using pd.merge with left_index=True and right_index=True.
# Check that the 'VIX_Close' column is present in df_analysis after the merge.
# Verify that the 'VIX_Regime_Above_Threshold' column is calculated based on 'VIX_Close' and the vix_threshold.
# Inspect the analyzed_data dictionary after the execution of cell fe783563 to confirm that the DataFrames for each asset contain the 'VIX_Close' and 'VIX_Regime_Above_Threshold' columns.

# The previous execution of cell fe783563 already ran the code.
# We can now inspect the analyzed_data dictionary to verify the changes.

# Check the columns of a sample DataFrame from analyzed_data
# For example, check the 'ETF' group and 'SPY' ticker
if 'analyzed_data' in locals() or 'analyzed_data' in globals():
    if 'ETF' in analyzed_data and 'SPY' in analyzed_data['ETF']:
        spy_df = analyzed_data['ETF']['SPY']
        print("\nColumns in analyzed_data['ETF']['SPY']:")
        print(spy_df.columns.tolist())

        print("\nTail of analyzed_data['ETF']['SPY']:")
        display(spy_df.tail())

        # You can also check another sample
        if 'STOCK' in analyzed_data and 'AAPL' in analyzed_data['STOCK']:
            aapl_df = analyzed_data['STOCK']['AAPL']
            print("\nColumns in analyzed_data['STOCK']['AAPL']:")
            print(aapl_df.columns.tolist())

            print("\nTail of analyzed_data['STOCK']['AAPL']:")
            display(aapl_df.tail())
    else:
        print("Error: 'analyzed_data' does not contain expected data.")
else:
    print("Error: 'analyzed_data' variable not found. Please ensure the previous cell ran successfully.")



Columns in analyzed_data['ETF']['SPY']:
['Open', 'Low', 'High', 'Close', 'Volume', 'RSI_14', 'ATRr_14', 'SMA_50', 'SMA_100', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'BBL_20_2.0_2.0', 'BBM_20_2.0_2.0', 'BBU_20_2.0_2.0', 'BBB_20_2.0_2.0', 'BBP_20_2.0_2.0', 'EMA_20', 'VWAP_D', 'STOCHk_14_3_3', 'STOCHd_14_3_3', 'STOCHh_14_3_3', 'STOCHk_vs_SMA50_Above', 'Fib_0_Retracement_Daily', 'Fib_23.6_Retracement_Daily', 'Fib_38.2_Retracement_Daily', 'Fib_50.0_Retracement_Daily', 'Fib_61.8_Retracement_Daily', 'Fib_78.6_Retracement_Daily', 'Fib_100_Retracement_Daily', 'Fib_123.6_Extension_Daily', 'Fib_138.2_Extension_Daily', 'Fib_161.8_Extension_Daily', 'Fib_200.0_Extension_Daily', 'Fib_261.8_Extension_Daily', 'Fib_0_Retracement_Weekly', 'Fib_23.6_Retracement_Weekly', 'Fib_38.2_Retracement_Weekly', 'Fib_50.0_Retracement_Weekly', 'Fib_61.8_Retracement_Weekly', 'Fib_78.6_Retracement_Weekly', 'Fib_100_Retracement_Weekly', 'Fib_123.6_Extension_Weekly', 'Fib_138.2_Extension_Weekly', 'Fib_161.8_Ext

,Open,Low,High,Close,Volume,RSI_14,ATRr_14,SMA_50,SMA_100,MACD_12_26_9,...,Fib_61.8_Retracement_Weekly,Fib_78.6_Retracement_Weekly,Fib_100_Retracement_Weekly,Fib_123.6_Extension_Weekly,Fib_138.2_Extension_Weekly,Fib_161.8_Extension_Weekly,Fib_200.0_Extension_Weekly,Fib_261.8_Extension_Weekly,VIX_Close,VIX_Regime_Above_Threshold
Date,,,,,,,,,,,,,,,,,,,,,
2025-09-23,663.210022,661.979980,667.340027,666.719971,81708900,73.713522,5.535953,640.456583,616.503841,7.269657,...,550.977849,519.345412,479.051713,711.776069,739.266163,783.702205,855.628341,971.990518,16.639999,0
2025-09-24,661.099976,659.669983,664.609985,664.510010,68082200,69.449397,5.644098,641.231082,617.577205,7.300283,...,550.977849,519.345412,479.051713,711.776069,739.266163,783.702205,855.628341,971.990518,16.180000,0
2025-09-25,658.049988,654.409973,659.409973,657.940002,89622100,58.597034,5.962380,641.949573,618.541519,6.716981,...,550.977849,519.345412,479.051713,711.776069,739.266163,783.702205,855.628341,971.990518,16.740000,0
2025-09-26,661.820007,657.880005,662.369995,659.510010,69179200,60.197643,5.857209,642.686299,619.543009,6.308673,...,550.977849,519.345412,479.051713,711.776069,739.266163,783.702205,855.628341,971.990518,15.290000,0
2025-09-29,663.679993,661.859985,665.280029,664.359985,73415500,64.733308,5.850981,643.422296,620.639134,6.303773,...,550.977849,519.345412,479.051713,711.776069,739.266163,783.702205,855.628341,971.990518,16.120001,0



Columns in analyzed_data['STOCK']['AAPL']:
['Open', 'Low', 'High', 'Close', 'Volume', 'RSI_14', 'ATRr_14', 'SMA_50', 'SMA_100', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'BBL_20_2.0_2.0', 'BBM_20_2.0_2.0', 'BBU_20_2.0_2.0', 'BBB_20_2.0_2.0', 'BBP_20_2.0_2.0', 'EMA_20', 'VWAP_D', 'STOCHk_14_3_3', 'STOCHd_14_3_3', 'STOCHh_14_3_3', 'STOCHk_vs_SMA50_Above', 'Fib_0_Retracement_Daily', 'Fib_23.6_Retracement_Daily', 'Fib_38.2_Retracement_Daily', 'Fib_50.0_Retracement_Daily', 'Fib_61.8_Retracement_Daily', 'Fib_78.6_Retracement_Daily', 'Fib_100_Retracement_Daily', 'Fib_123.6_Extension_Daily', 'Fib_138.2_Extension_Daily', 'Fib_161.8_Extension_Daily', 'Fib_200.0_Extension_Daily', 'Fib_261.8_Extension_Daily', 'Fib_0_Retracement_Weekly', 'Fib_23.6_Retracement_Weekly', 'Fib_38.2_Retracement_Weekly', 'Fib_50.0_Retracement_Weekly', 'Fib_61.8_Retracement_Weekly', 'Fib_78.6_Retracement_Weekly', 'Fib_100_Retracement_Weekly', 'Fib_123.6_Extension_Weekly', 'Fib_138.2_Extension_Weekly', 'Fib_161.8_

,Open,Low,High,Close,Volume,RSI_14,ATRr_14,SMA_50,SMA_100,MACD_12_26_9,...,Fib_61.8_Retracement_Weekly,Fib_78.6_Retracement_Weekly,Fib_100_Retracement_Weekly,Fib_123.6_Extension_Weekly,Fib_138.2_Extension_Weekly,Fib_161.8_Extension_Weekly,Fib_200.0_Extension_Weekly,Fib_261.8_Extension_Weekly,VIX_Close,VIX_Regime_Above_Threshold
Date,,,,,,,,,,,,,,,,,,,,,
2025-09-23,254.429993,253.580002,257.339996,255.880005,60275200,76.047586,6.503082,225.434800,214.538845,5.883976,...,202.719649,187.800717,168.796839,278.557553,291.522816,312.480363,346.403173,401.28353,16.639999,0
2025-09-24,252.309998,251.039993,255.740005,255.220001,42303700,74.841917,6.384291,226.359543,215.005350,6.600403,...,202.719649,187.800717,168.796839,278.557553,291.522816,312.480363,346.403173,401.28353,16.180000,0
2025-09-25,256.869995,251.710007,257.170013,253.210007,55202100,71.142726,6.318271,227.222511,215.481582,6.926146,...,202.719649,187.800717,168.796839,278.557553,291.522816,312.480363,346.403173,401.28353,16.740000,0
2025-09-26,255.460007,253.779999,257.600006,254.100006,46076300,71.807202,6.180537,228.097885,215.996542,7.173424,...,202.719649,187.800717,168.796839,278.557553,291.522816,312.480363,346.403173,401.28353,15.290000,0
2025-09-29,254.429993,253.009995,255.000000,254.559998,40091100,72.163962,5.881213,228.976466,216.564881,7.322107,...,202.719649,187.800717,168.796839,278.557553,291.522816,312.480363,346.403173,401.28353,16.120001,0


## Summary:

### Data Analysis Key Findings

*   Historical VIX data for the `^VIX` ticker was successfully fetched for the specified time range (150 days) using `yfinance`.
*   The fetched VIX data was merged with the historical data for each individual asset (ETFs and stocks) in the `analyzed_data` dictionary.
*   A new column, 'VIX\_Close', containing the daily closing price of the VIX, was added to the DataFrame of each asset.
*   A 'VIX\_Regime\_Above\_Threshold' indicator was calculated for each asset, a binary variable indicating whether the VIX closing price was above a threshold of 20.
*   The `analyzed_data` DataFrames for each asset now include both the VIX closing prices and the calculated VIX Regime indicator.

### Insights or Next Steps

*   The VIX data and regime indicator can now be used in further analysis to understand how asset price movements correlate with changes in market volatility.
*   Explore implementing more sophisticated VIX regime definitions, potentially using moving averages, standard deviations, or different threshold levels based on historical VIX behavior.
